In [6]:
insert_query = '''
INSERT INTO students (name, age, class, section, english, hindi, science)
VALUES (%s, %s, %s, %s, %s, %s, %s)
'''

# Step 4: Iterate over the rows of the dataframe and insert data
for index, row in data.iterrows():
    cursor.execute(insert_query, (
        row['name'],
        row['age'],
        row['class'],
        row['section'],
        row['english'],
        row['hindi'],
        row['science']
    ))

# Step 5: Commit the transaction
conn.commit()


In [3]:
csv_file = 'databasetable.csv'
data = pd.read_csv(csv_file)

In [1]:
import mysql.connector
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import spacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Span
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Anudeep
[nltk_data]     Bura\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:

df=pd.read_csv("studentdatset.csv")
# Prepare data

queries=df["Query"]
intents=df["Intent"]
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(intents)

# Split data
X_train, X_test, y_train, y_test = train_test_split(queries, y, test_size=0.2, random_state=42)

# Create and train model
model = make_pipeline(TfidfVectorizer(), LogisticRegression())
model.fit(X_train, y_train)

# Predict intent
def predict_intent(query):
    intent_idx = model.predict([query])[0]
    return label_encoder.inverse_transform([intent_idx])[0]



In [3]:
#rule-based matching of text patterns
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the matcher
matcher = Matcher(nlp.vocab)
dbcolumns=["age","name","section","class","id","english","hindi","science"]
patterns=[]
for colm in dbcolumns:
    patterns.append({"label": colm.upper(), "pattern": [{"LOWER": colm}, {"IS_ALPHA":True}]})
    patterns.append({"label": colm.upper(), "pattern": [{"LOWER": colm}, {"IS_DIGIT":True}]})


# Add patterns to the matcher
for pattern in patterns:
    matcher.add(pattern["label"], [pattern["pattern"]])

# Initialize the phrase matcher for known entities
phrase_matcher = PhraseMatcher(nlp.vocab)
#Aho-Corasick algorithm
known_entities = [ "english", "hindi", "science"]
patterns = [nlp(text) for text in known_entities]
phrase_matcher.add("KNOWN_ENTITIES", None, *patterns)

def extract_and_print_entities(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Extract entities using NER
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    # Extract entities using rule-based matching
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        entities.append((span.text, nlp.vocab.strings[match_id]))
    
    # Extract entities using phrase matching
    phrase_matches = phrase_matcher(doc)
    for match_id, start, end in phrase_matches:
        span = doc[start:end]
        entities.append((span.text, "KNOWN_ENTITY"))
    
    # Print entities
    if entities:
        print("Entities found in the text:")
        for entity in entities:
            print(f"Text: {entity[0]}, Label: {entity[1]}")
        return entities
    else:
        print("No entities found in the text.")


In [4]:
def preprocess(naturallang):
    pattern = r'(\d+)(st|nd|rd|th)'
    naturallang=re.sub(pattern, r'\1', naturallang)
    naturallang=naturallang.replace("grade","class")
    words = nltk.word_tokenize(naturallang)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)
    words = word_tokenize(lemmatized_text)
    stop_words = set(stopwords.words('english'))
    stop_words.remove("a")
    filtered_words = [word for word in words if word.lower() not in stop_words]
    gg=' '.join(filtered_words)
    gg=gg.replace('id','ID')
    g=gg.replace("'","").replace(",","").replace("  "," ")
    return g

In [5]:
naturallang="Add a new student with name 'Vera', age 15, grade 10, section 'B'"
naturallang=preprocess(naturallang)
print(naturallang)
intent=predict_intent(naturallang)
entities=extract_and_print_entities(naturallang)


Add a new student name Vera  age 15 class 10 section B 
Entities found in the text:
Text: Vera, Label: ORG
Text: age 15, Label: DATE
Text: name Vera, Label: NAME
Text: age 15, Label: AGE
Text: class 10, Label: CLASS
Text: section B, Label: SECTION


In [6]:
print(naturallang)
columns=["CLASS","SECTION","NAME","NAMES","ID","AGE","ENGLISH","HINDI","SCIENCE","GRADE"]
req=[]
for entitie in entities:
    if(entitie[1] in columns):
        req.append(entitie)
print(req)

Add a new student name Vera  age 15 class 10 section B 
[('name Vera', 'NAME'), ('age 15', 'AGE'), ('class 10', 'CLASS'), ('section B', 'SECTION')]


In [7]:
req[0][0]

'name Vera'

In [8]:
finalcolm=[]
finalcondi=[]
for i in req:
    if " " in i[0]:
        finalcondi.append(i[0].replace(" ","="))
    else:
        finalcolm.append(i[0])
print(finalcondi)
print(finalcolm)

['name=Vera', 'age=15', 'class=10', 'section=B']
[]


In [9]:

query=""
if intent=="SELECT":
    cols=""
    condi=""
    for i in finalcolm:
        cols+=i

        if i!=len(finalcolm-1):
            cols+=","
    for j in range(len(finalcondi)-1):
        print(j)
        k=finalcondi[j].split("=")
        cols+=k[0]
        
        if j!=len(finalcondi)-2:
            cols+=","
    con=finalcondi[len(finalcondi)-1].split("=")
    condi+=con[0]
    if con[1].isdigit():
        print(con[1].isdigit)
        
        condi+="="+con[1]
    else:
        condi+="=\'"+con[1]+"\'"

    query="SELECT "+cols+" FROM students"
    if condi!="":
        query+=" WHERE "+condi+";"
elif intent=="UPDATE":
    colnames=""
    val=""
    for i in range(0,len(finalcondi)):
        if i==len(finalcondi)-1:
            i=finalcondi[i].split("=")
            if i[1].isdigit:
                colnames+=i[0]+"="+i[1]
            else:
                colnames+=i[0]+"="+"\'"+i[1]+"\'"
        elif i==len(finalcondi)-2:
            i=finalcondi[i].split("=")
            if i[1].isdigit:
                val+=i[0]+"="+i[1]
            else:
                val+=i[0]+"="+"\'"+i[1]+"\'"
        else:
            i=finalcondi[i].split("=")
            if i[1].isdigit():
                val+=i[0]+"="+i[1]+","
            else:
                val+=i[0]+"="+"\'"+i[1]+"\'"+","
    query="UPDATE students SET "+ val+ " WHERE "+ colnames+";"
elif intent=="DELETE":
    gg=""
    for i in finalcondi:
        i=i.split("=")
        if i[1].isdigit():
            gg+=i[0]+"="+i[1]
        else:
            gg+=i[0]+"="+"\'"+i[1]+"\'"
    query="DELETE FROM students WHERE "+gg+";"

elif intent=="INSERT":
    colnames=""
    val=""
    for u in range(0,len(finalcondi)):
        if u==len(finalcondi)-1:
            g=finalcondi[u].split("=")
            colnames+=g[0]
            if g[1].isdigit():
                val+=g[1]+","
            else:
                val+="\'"+g[1]+"\'"+","
        else:
            g=finalcondi[u].split("=")
            colnames+=g[0]+","
            if g[1].isdigit():
                val+=g[1]+","
            else:
                val+="\'"+g[1]+"\'"+","
    query="INSERT INTO students ("+colnames+") VALUES ("+val+");"
print(query)
        


INSERT INTO students (name,age,class,section) VALUES ('Vera',15,10,'B',);


In [58]:
# Replace with your actual database credentials
conn = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or '127.0.0.1'
    user='root',    # e.g., 'root'
    password='Marsmover$123',# e.g., 'your_password'
    database='school'        # Your database name
)

cursor = conn.cursor()
cursor.execute("SELECT section FROM students WHERE name='John Doe'")
cursor.fetchall()


[('A',)]

In [ ]:
conn.commit()

In [ ]:
cursor.close()
conn.close()